<a href="https://colab.research.google.com/github/Trung0Minh/AIO2023-MODULE-3/blob/main/statistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
def cosine_similarity(x, y):
    numerator = np.dot(x, y)
    denominator = np.linalg.norm(x) * np.linalg.norm(y)
    return numerator / denominator

In [45]:
def window_based_matching_cosine(left_img, right_img, disparity_range, kernel_size):
    left = cv2.imread(left_img, 0)
    right = cv2.imread(right_img, 0)

    left = left.astype(np.float32)
    right = right.astype(np.float32)

    height, width = left.shape

    depth = np.zeros((height, width), np.uint8)
    kernel_half = kernel_size // 2
    scale = 3

    for y in range(kernel_half, height - kernel_half):
        for x in range(kernel_half, width - kernel_half):
            disparity = 0
            cost_optimal = -1
            for d in range(disparity_range):
                dist = x - d - kernel_half
                cost = -1
                if dist > 0:
                    cost = cosine_similarity(left[y - kernel_half:y + kernel_half + 1, x - kernel_half:x + kernel_half + 1].flatten(),
                                             right[y - kernel_half:y + kernel_half + 1, x - kernel_half - d:x + kernel_half + 1 - d].flatten())
                    if cost > cost_optimal:
                        cost_optimal = cost
                        disparity = d

            depth[y, x] = disparity * scale

    return depth

In [ ]:
def compute_rms_error(matrix1, matrix2):
    return np.sqrt(np.mean(np.square(matrix1 - matrix2)))

In [46]:
left_img_path = 'view1_d.png'
right_img_path = 'view5_d.png'
groundtruth_img_path = 'disp1.png'

disparity_range = 64
kernel_size = 5

depth = window_based_matching_cosine(left_img_path, right_img_path, disparity_range, kernel_size)

groundtruth = cv2.imread(groundtruth_img_path, 0)
rmse = compute_rms_error(depth, groundtruth)
print("RMSE:", rmse)

RMSE: 5.605343375188772


In [ ]:
def compute_correlation_coefficient(X, Y):
    N = len(X)
    numerator = np.mean((X - np.mean(X)) * (Y - np.mean(Y)))
    denominator = np.std(X) * np.std(Y)
    return np.round(numerator / denominator, 2)

In [ ]:
def window_based_matching_correlation(left_img, right_img, disparity_range, kernel_size):
    left = cv2.imread(left_img, 0)
    right = cv2.imread(right_img, 0)

    left = left.astype(np.float32)
    right = right.astype(np.float32)

    height, width = left.shape

    depth = np.zeros((height, width), np.uint8)
    kernel_half = kernel_size // 2
    scale = 3

    for y in range(kernel_half, height - kernel_half):
        for x in range(kernel_half, width - kernel_half):
            disparity = 0
            cost_optimal = -1
            for d in range(disparity_range):
                dist = x - d - kernel_half
                cost = -1
                if dist > 0:
                    cost = compute_correlation_coefficient(left[y - kernel_half:y + kernel_half + 1, x - kernel_half:x + kernel_half + 1],
                                             right[y - kernel_half:y + kernel_half + 1, x - kernel_half - d:x + kernel_half + 1 - d])
                    if cost > cost_optimal:
                        cost_optimal = cost
                        disparity = d

            depth[y, x] = disparity * scale

    return depth

In [47]:
depth = window_based_matching_correlation(left_img_path, right_img_path, disparity_range, kernel_size)
rmse = compute_rms_error(depth, groundtruth)
print("RMSE:", rmse)

<ipython-input-40-b998f2eea59c>:5: RuntimeWarning: invalid value encountered in scalar divide
  return np.round(numerator / denominator, 2)


RMSE: 5.38587686965895
